# IBM Capstone Project: Segmenting & Clustering Neighborhoods in Toronto pt.2/2

In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import msgpack
import json
import requests
from bs4 import BeautifulSoup
import csv
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
done=print('done!')
done

done!


In [2]:
# install required libraries
!pip install folium
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge geocoder -y

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=

In [3]:
# From pt. 1, create the pandas dataframe
link=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(link.text,'lxml')


# write into a csv file
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
csv_file=open('toronto_1.csv','w')
csv_write=csv.writer(csv_file)
csv_write.writerow(['PostalCode','Borough','Neighborhood'])
for tr in soup.find_all('tr')[1:]:
    td_all = tr.find_all('td')
    if len(td_all) == 3:
        PostalCode=td_all[0].text
        Borough=td_all[1].text
        Neighborhood=td_all[2].text
    csv_write.writerow([PostalCode,Borough,Neighborhood])
csv_file.close()

# assign to a pandas dataframe
df_toronto=pd.read_csv('toronto_1.csv')

## clean the data

# remove \n enters
df_toronto[df_toronto.columns] =df_toronto.apply(lambda x: x.str.strip('\n'))
# remove all rows with 'Not assigned' Boroughs
df_toronto_cleaned = df_toronto[df_toronto.Borough!='Not assigned']
#preview shape
df_toronto_cleaned.shape

(212, 3)

In [4]:
# preview
df_toronto_cleaned.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
# reset index
df_toronto_cleaned=df_toronto_cleaned.reset_index()
df_toronto_cleaned

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights
5,7,M6A,North York,Lawrence Manor
6,8,M7A,Queen's Park,Not assigned
7,10,M9A,Etobicoke,Islington Avenue
8,11,M1B,Scarborough,Rouge
9,12,M1B,Scarborough,Malvern


In [13]:
df_toronto_cleaned.drop('index',axis=1,inplace=True)
df_toronto_cleaned

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Question 2: Get geographical lattitudes & longitudes based on a given postal code

In [15]:
# collect csv file of coordinates
link="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
coordinates=pd.read_csv(link)
coordinates.columns=['PostalCode','Latittude','Longitude']

# merge into a pandas dataframe
df_toronto_coordinates = pd.merge(df_toronto_cleaned, on="PostalCode")
df_toronto_coordinates = df_toronto_coordinates[df_toronto_coordinates['Borough'].str.contains('Toronto')].reset_index(drop=True)

# Preview
df_toronto_coordinates.head()

TypeError: merge() missing 1 required positional argument: 'right'